3.1 Image Formation Model

In [1]:
import numpy as np
import cv2

def image_formation_model( f, x0, y0, sigma ):
    g = f.copy( )
    nr, nc = f.shape[:2]
    illumination = np.zeros( [ nr, nc ], dtype = 'float32' )
    for x in range( nr ):
        for y in range( nc ):
            illumination[x,y] = np.floor(255*np.exp( -( ( x - x0 ) ** 2 + ( y - y0 ) ** 2 ) / 
                                 ( 2 * sigma * sigma ) ))
    for x in range( nr ):
        for y in range( nc ):
            for k in range( 3 ):
                val = round( illumination[x,y] * f[x,y,k] )
                g[x,y,k] = np.uint8( val )
    return illumination

def main( ):
    img = cv2.imread( "Monet.bmp", -1 )
    nr, nc = img.shape[:2]
    x0 = nr/2+200
    y0 = nc/2
    sigma = 100
    img2 = image_formation_model( img, x0, y0, sigma )
    cv2.imwrite( "Monet_illumination.bmp", img2 )  

main( )

3.2 Image Downsampling

In [6]:
import numpy as np
import cv2

def image_downsampling( f, sampling_rate ):
    nr, nc = f.shape[:2]
    nr_s, nc_s = nr // sampling_rate, nc // sampling_rate
    g = np.zeros( [ nr_s, nc_s ], dtype = 'uint8' )
    for x in range( nr_s ):
        for y in range( nc_s ):
            g[x,y] = f[x * sampling_rate, y * sampling_rate]
    return g

def main( ):
    img1 = cv2.imread( "Barbara.bmp", -1 )
    img2 = image_downsampling( img1, 4 )
    cv2.imwrite( "Barbara_Down.bmp", img2 )  

main( )

3.3 Image Quantization

In [7]:
import numpy as np
import cv2

def image_quantization( f, bits ):
    g = f.copy( )
    nr, nc = f.shape[:2]
    levels = 2 ** bits
    interval = 256 / levels
    gray_level_interval = 255 / ( levels - 1 )
    table = np.zeros( 256 )
    for k in range( 256 ):
        for l in range( levels ):
            if k >= l * interval and l < ( l + 1 ) * interval:
                table[k] = round( l * gray_level_interval ) 
    for x in range( nr ):
        for y in range( nc ):
            g[x,y] = np.uint8( table[f[x,y]] )
    return g

def main( ):
    img1 = cv2.imread( "Lenna.bmp", -1 )
    img2 = image_quantization( img1, 3 )
    cv2.imwrite( "Lenna_Quan.bmp", img2 )  

main( )

<font color="Red">3.4 Practice Complements(gray) </font>

In [32]:
import numpy as np
import cv2

def image_complement( f ):
    nr, nc = f.shape[:2]
    g = np.zeros( [ nr, nc ], dtype = 'uint8' )
    g = 255-f
    return g

def image_union( f, g ):
    nr, nc = f.shape[:2]
    u = np.zeros( [ nr, nc ], dtype = 'uint8' )
    for x in range(nr):
        for y in range(nc):
            u[x,y] = max(g[x,y],f[x,y])
    return u

def main( ):
    img1 = cv2.imread( "Barbara.bmp", -1 )
    img2 = image_complement( img1 )
    cv2.imwrite( "Barbara_Complement.bmp", img2 )  
    img3 = image_union( img1, img2 )
    cv2.imwrite( "Barbara_Union.bmp", img3 )  

main( )

<font color="Red">3.5 Practice Complements(color) </font>

In [1]:
import numpy as np
import cv2

def image_complement( f ):
    nr, nc = f.shape[:2]
    g = np.zeros( [ nr, nc , 3], dtype = 'uint8' )
    g = 255-f
    return g

def image_union( f, g ):
    nr, nc = f.shape[:2]
    u = np.zeros( [ nr, nc ,3], dtype = 'uint8' )
    for z in range(3):
        for x in range(nr):
            for y in range(nc):
                u[x,y,z] = max(g[x,y,z],f[x,y,z])
    return u

def main( ):
    img1 = cv2.imread( "Baboon.bmp", -1 )
    img2 = image_complement( img1 )
    cv2.imwrite( "Baboon_Complement.bmp", img2 )  
    img3 = image_union( img1, img2 )
    cv2.imwrite( "Baboon_Union.bmp", img3 )  

main( )

3.6 Image Scaling

In [8]:
import numpy as np
import cv2

img1 = cv2.imread( "Lenna.bmp", -1 )
nr, nc = img1.shape[:2]
scale = eval( input( "Please enter scale: " ) )
nr2 = int( nr * scale )
nc2 = int( nc * scale )
img2 = cv2.resize( img1, ( nr2, nc2 ), interpolation = cv2.INTER_LINEAR )
cv2.imwrite( "Lenna_ori.bmp", img1 )  
cv2.imwrite( "Lenna_scaling.bmp", img2 )  


Please enter scale: 1.5


True

interpolation 選項所用的插值方法
INTER_NEAREST 最近鄰插值
INTER_LINEAR 雙線性插值（默認設置）
INTER_AREA 使用像素區域關係進行重採樣。可能是圖像抽取的首選方法，因它會產生無雲紋理的結果。但是當圖像縮放時，它類似於INTER_NEAREST方法。
INTER_CUBIC 4x4像素鄰域的雙三次插值
INTER_LANCZOS4 8x8像素鄰域的Lanczos插值

3.7 Image Rescaling

In [4]:
import numpy as np
import cv2

img = cv2.imread( "Baboon.bmp", 0 )
nr1, nc1 = img.shape[:2]
nr2, nc2 = nr1 // 4, nc1 // 4
img1 = cv2.resize( img, ( nr2, nc2 ), interpolation = cv2.INTER_NEAREST )
img1 = cv2.resize( img1, ( nr1, nc1 ), interpolation = cv2.INTER_NEAREST )
img2 = cv2.resize( img, ( nr2, nc2 ), interpolation = cv2.INTER_LINEAR )
img2 = cv2.resize( img2, ( nr1, nc1 ), interpolation = cv2.INTER_LINEAR )
img3 = cv2.resize( img, ( nr2, nc2 ), interpolation = cv2.INTER_CUBIC )
img3 = cv2.resize( img3, ( nr1, nc1 ), interpolation = cv2.INTER_CUBIC )
cv2.imwrite( "Baboon_ori.bmp", img ) 
cv2.imwrite( "Baboon_NN.bmp", img1 )  
cv2.imwrite( "Baboon_Bilinear.bmp", img2 )  
cv2.imwrite( "Baboon_Bicubic.bmp", img3 )  


True

<font color="Red">Practice 3-8: PSNR</font> <p>
Design a PSNR function to measure the difference between 
(img,img1), (img,img2), and (img,img3)<p>
<img src="https://4.bp.blogspot.com/-82C6B1wcYz0/UvIjcJ4S6TI/AAAAAAAABCY/OwGVIQwgUAc/s1600/fig1.PNG"></p>

In [5]:
import numpy as np
import math
import cv2

def psnr(I, K):
    MSE=0
    MSE=np.float64(MSE)
    PSNR=0
    PSNR=np.float64(PSNR)
    sum=0
    sum=np.float64(sum)
    m, n = img1.shape[:2]
    for i in range(m):
        for j in range(n):
            sum += np.power((np.float64(I[i,j]) - np.float64(K[i,j])),2)
    MSE = sum / (m*n)
    PSNR=10*np.log10(np.power(255,2)/MSE)                     
    return PSNR

img = cv2.imread( "Baboon_ori.bmp", 0)
img1 = cv2.imread("Baboon_NN.bmp",0)
img2 = cv2.imread("Baboon_Bilinear.bmp",0)
img3 = cv2.imread("Baboon_Bicubic.bmp",0)

psnr1 = psnr(img,img1)
psnr2 = psnr(img,img2)
psnr3 = psnr(img,img3)
print("PSNR between origin and Nearest Neighbor",psnr1)
print("PSNR between origin and Bilinear",psnr2)
print("PSNR between origin and Bicubic",psnr3)

PSNR between origin and Nearest Neighbor 17.57302656330855
PSNR between origin and Bilinear 20.55827625676485
PSNR between origin and Bicubic 19.49410245097318


In [31]:
import numpy as np
import math
n=1
n=np.log10(n)
print(type(n))
n=np.power(n,2)
print(type(n))
n=math.pow(n,2)
print(type(n))
n=1
n=math.log(n,10)
print(type(n))

<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'float'>
<class 'float'>


3.9 Image Rotation

In [40]:
import numpy as np
import cv2

img1 = cv2.imread( "Lenna.bmp", -1 )
nr2, nc2 = img1.shape[:2]
rotation_matrix = cv2.getRotationMatrix2D( ( nr2 / 2, nc2 / 2 ), 45, 1 )
img2 = cv2.warpAffine( img1, rotation_matrix, ( nr2, nc2 ) ) 
cv2.imwrite( "Lenna_Rotation.bmp", img2)  

True

3.10 Image Flip

In [41]:
import numpy as np
import cv2

img = cv2.imread( "Baboon.bmp", -1 )
img1 = cv2.flip( img, 0 )
img2 = cv2.flip( img, 1 )
img3 = cv2.flip( img, -1 )
cv2.imwrite( "Baboon_flip1.bmp", img1) 
cv2.imwrite( "Baboon_flip2.bmp", img2) 
cv2.imwrite( "Baboon_flip3.bmp", img3) 

True

3.11 Perspective Transform

In [67]:
import numpy as np
import cv2

img1 = cv2.imread( "Gallery.bmp", -1 )
nr, nc = img1.shape[:2]
pts1 = np.float32( [ [ 795, 350 ], [ 795, 690 ], [ 1090, 720 ], [ 1090, 250 ] ] )
pts2 = np.float32( [ [ 0, 0 ], [ 0, 500 ], [ 650, 500 ], [ 650, 0 ] ] )
T = cv2.getPerspectiveTransform( pts1, pts2 )
img2 = cv2.warpPerspective( img1, T, ( 650, 500 ) )

cv2.imwrite( "Gallery_perspec.bmp", img2 )


True

<font color="Red">3.12 Practice:Perspective transform </font>
<img src="Gallery.bmp" width="200px" height="200px">
<img src="Gallery_perspec_hw.bmp" width="200px" height="200px">

In [72]:
import numpy as np
import cv2

img1 = cv2.imread( "Gallery.bmp", -1 )
nr, nc = img1.shape[:2]
pts1 = np.float32( [ [ 1425, 572 ], [ 1425, 1109 ], [ 1824, 1222 ], [ 1824, 552 ] ] )
'''小畫家抓點'''
pts2 = np.float32( [ [ 0, 0 ], [ 0, 500 ], [ 650, 500 ], [ 650, 0 ] ] )
T = cv2.getPerspectiveTransform( pts1, pts2 )
img2 = cv2.warpPerspective( img1, T, ( 650, 500 ) )

cv2.imwrite( "Gallery_perspec_HHW.bmp", img2 )

True

<script type="text/javascript" src="ASCIIMathML.js"> </script>


<font color="Red">3.13 </font>

<!DOCTYPE html><html><head><title>MathJax TeX Test Page</title><script type="text/x-mathjax-config">  MathJax.Hub.Config({tex2jax: {inlineMath: [['$','$'], ['\\(','\\)']]}});</script><script type="text/javascript"  src="http://cdn.mathjax.org/mathjax/latest/MathJax.js?config=TeX-AMS-
    MML_HTMLorMML"></script></head>
<p>$$inverse \begin{bmatrix}
X\\
Y\\
\end{bmatrix} =\begin{bmatrix}
X'\\
Y'\\
\end{bmatrix}
\begin{bmatrix}
cos\theta& sin\theta \\
-sin\theta&cos\theta\\
\end{bmatrix}
$$</p>
<p>$$Ori$$<p/>
<p>$$X'=(X-X_0)cos\theta-(Y-Y_0)sin\theta$$</p><p>$$Y' = (X-X_0)sin\theta+(Y-Y_0)cos\theta$$</p></html>
<p>$$inverse$$<p/>
<p>$$X-X_0=X'cos\theta+Y'sin\theta$$</p><p>$$Y-Y_0=-X'sin\theta-Y'cos\theta$$</p></html>